![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/visual-nlp/2.1.Pdf_processing.ipynb)

## Blogposts and videos

- [Text Detection in Spark OCR](https://medium.com/spark-nlp/text-detection-in-spark-ocr-dcd8002bdc97)

- [Table Detection & Extraction in Spark OCR](https://medium.com/spark-nlp/table-detection-extraction-in-spark-ocr-50765c6cedc9)

- [Extract Tabular Data from PDF in Spark OCR](https://medium.com/spark-nlp/extract-tabular-data-from-pdf-in-spark-ocr-b02136bc0fcb)

- [Signature Detection in Spark OCR](https://medium.com/spark-nlp/signature-detection-in-spark-ocr-32f9e6f91e3c)

- [GPU image pre-processing in Spark OCR](https://medium.com/spark-nlp/gpu-image-pre-processing-in-spark-ocr-3-1-0-6fc27560a9bb)

- [How to Setup Spark OCR on UBUNTU - Video](https://www.youtube.com/watch?v=cmt4WIcL0nI)


**More examples here**

https://github.com/JohnSnowLabs/spark-ocr-workshop

For get the trial license please go to:

https://www.johnsnowlabs.com/install/

### Colab Setup

In [1]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
!pip install -q johnsnowlabs==5.1.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 643.8/643.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.3/531.3 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 8.9 MB/s eta 

In [2]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

Please Upload your John Snow Labs License using the button below


Saving OCT_23_Visual_NLP.json to OCT_23_Visual_NLP.json


In [ ]:
from johnsnowlabs import nlp, visual

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.install(refresh_install=True, visual=True)

In [4]:
from johnsnowlabs import nlp, visual
import pandas as pd

# Automatically load license data and start a session with all jars user has access to
spark = nlp.start(visual=True)

👌 Detected license file /content/OCT_23_Visual_NLP.json
👌 Launched cpu optimized session with with: 🚀Spark-NLP==5.1.1, 🕶Spark-OCR==5.0.2, running on ⚡ PySpark==3.1.2


In [5]:
import pkg_resources

from pyspark.ml import PipelineModel
from pyspark.sql import functions as F

## Read pdfs to the dataframe and display

In [6]:
pdf_path = visual.pkg_resources.resource_filename('sparkocr', 'resources/ocr/pdfs/*.pdf')
pdf_example_df = spark.read.format("binaryFile").load(pdf_path).cache()

visual.display_pdf(pdf_example_df)

Output hidden; open in https://colab.research.google.com to view.

## Define pipeline for extract text from the searchable and ocr text from the scaned pdf

In [7]:
def pipeline():

    # If text PDF extract text
    pdf_to_text = visual.PdfToText() \
        .setInputCol("content") \
        .setOutputCol("text") \
        .setSplitPage(True) \
        .setExtractCoordinates(True) \
        .setStoreSplittedPdf(True)

    # If image pdf, extract image
    pdf_to_image = visual.PdfToImage() \
        .setInputCol("content") \
        .setOutputCol("image") \
        .setKeepInput(True)

    # Run OCR
    ocr = visual.ImageToText() \
        .setInputCol("image") \
        .setOutputCol("text") \
        .setConfidenceThreshold(60)

    pipeline = PipelineModel(stages=[
        pdf_to_text,
        pdf_to_image,
        ocr
    ])

    return pipeline

## Run pipeline and show results

In [8]:
result = pipeline().transform(pdf_example_df).cache()
result.show()

+--------------------+--------------------+------+--------------------+--------------------+----------------+---------------+--------------------+--------------------+-----------+-------+-----------+--------------------+---------+
|                path|    modificationTime|length|                text|           positions|height_dimension|width_dimension|             content|               image|total_pages|pagenum|documentnum|          confidence|exception|
+--------------------+--------------------+------+--------------------+--------------------+----------------+---------------+--------------------+--------------------+-----------+-------+-----------+--------------------+---------+
|file:/usr/local/l...|2023-10-11 23:27:...| 70556|Alexandria is the...|[{[{A, 72.024, 76...|             792|            612|[25 50 44 46 2D 3...|                null|          0|      0|          0|-1.79769313486231...|     null|
|file:/usr/local/l...|2023-10-11 23:27:...|693743|Patient Nam
Finan...|[{[{P

## Display text using pandas dataframe

In [9]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

result.select("text").toPandas().style.set_properties(**{'white-space': 'pre-wrap', 'text-align': 'left'})